In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
# pretrained_model = "distilbert-base-uncased"
# pretrained_model = "xlm-roberta-large"
pretrained_model = "microsoft/deberta-v3-small" # not work
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "conll2003"
checkpoint_name = 'checkpoint-3512'
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}/{checkpoint_name}"

# use_pretrained_model = False
# if use_pretrained_model:
#     model_checkpoint = pretrained_model

batch_size = 16

# dataset_name = "wnut_17"
dataset_name = "SpeedOfMagic/ontonotes_english"


# Loading dataset

In [2]:
from datasets import load_dataset, load_metric

In [3]:
datasets = load_dataset(dataset_name)

Using custom data configuration SpeedOfMagic--ontonotes_english-955f75f6fda13fd9
Found cached dataset parquet (/root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13900
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8262
    })
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 59924
    })
})

In [5]:
datasets["train"][0]

{'tokens': ['Sunday', 'the', 'interview', 'with', 'Bob', 'Shapiro', '.'],
 'ner_tags': [15, 0, 0, 0, 1, 2, 0]}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=37, names=['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE'], id=None), length=-1, id=None)

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-PERSON',
 'I-PERSON',
 'B-NORP',
 'I-NORP',
 'B-FAC',
 'I-FAC',
 'B-ORG',
 'I-ORG',
 'B-GPE',
 'I-GPE',
 'B-LOC',
 'I-LOC',
 'B-PRODUCT',
 'I-PRODUCT',
 'B-DATE',
 'I-DATE',
 'B-TIME',
 'I-TIME',
 'B-PERCENT',
 'I-PERCENT',
 'B-MONEY',
 'I-MONEY',
 'B-QUANTITY',
 'I-QUANTITY',
 'B-ORDINAL',
 'I-ORDINAL',
 'B-CARDINAL',
 'I-CARDINAL',
 'B-EVENT',
 'I-EVENT',
 'B-WORK_OF_ART',
 'I-WORK_OF_ART',
 'B-LAW',
 'I-LAW',
 'B-LANGUAGE',
 'I-LANGUAGE']

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,tokens,ner_tags
0,"[it, 's, definitely, like]","[O, O, O, O]"
1,"[He, was, convicted, of, lewd, and, lascivious, acts, with, a, child, .]","[O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[They, talked, about, one, individual, and, one, group, which, we, know, the, name, of, that, was, deliberately, going, to, take, action, against, the, U.S., and, or, Israel, in, Yemen, ,, and, a, top, Pentagon, official, testifies, the, warning, was, actually, ,, ,, issued, only, hours, before, the, Cole, was, bombed, .]","[O, O, O, B-CARDINAL, O, O, B-CARDINAL, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-GPE, O, O, B-GPE, O, B-GPE, O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, B-TIME, I-TIME, O, O, B-PRODUCT, O, O, O]"
3,[Right],[O]
4,"[The, two, are, also, signing, a, trade, agreement, .]","[O, B-CARDINAL, O, O, O, O, O, O, O]"
5,"[We, should, understand, that, the, government, is, no, panacea, for, all, ills, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"[Is, n't, it, true, that, you, were, so, worked, up, that, framing, Mr., Laff, for, this, crime, was, the, least, that, you, planned, for, him, ?, '']","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PERSON, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,"[Texaco, ,, thinking, in, terms, of, $, 18, -, to, -, $, 19, oil, for, 1990, ,, may, raise, spending, ,, especially, for, low, -, risk, prospects, ,, an, official, says, .]","[B-ORG, O, O, O, O, O, B-MONEY, I-MONEY, I-MONEY, I-MONEY, I-MONEY, I-MONEY, I-MONEY, O, O, B-DATE, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,"[I, mean, you, know, what, a, terrible, thing, to, be, menopausal, .]","[O, O, O, O, O, O, O, O, O, O, O, O]"
9,"[Here, 's, CNN, 's, Cly, Wallace, .]","[O, O, B-ORG, O, B-PERSON, I-PERSON, O]"


# Preprocessing dataset

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
example = datasets["train"][4]
print(example["tokens"])

['and', 'they', 'are', 'in', 'Atlanta', '.']


In [14]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁and', '▁they', '▁are', '▁in', '▁Atlanta', '▁.', '[SEP]']


In [15]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(6, 8)

In [16]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, None]


In [17]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

8 8


In [18]:
label_all_tokens = True

In [19]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [20]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 1333, 262, 2109, 275, 3877, 31922, 323, 2], [1, 1864, 9977, 3330, 264, 8458, 1016, 4023, 323, 2], [1, 279, 1376, 271, 47701, 281, 40618, 10637, 263, 6654, 15773, 323, 2], [1, 272, 382, 268, 6654, 277, 262, 595, 40618, 277, 262, 423, 323, 2], [1, 263, 306, 281, 267, 5100, 323, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 15, 0, 0, 0, 1, 2, 0, -100], [-100, 31, 31, 0, 0, 0, 0, 17, 0, -100], [-100, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, -100], [-100, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 9, 0, -100]]}

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c868c86fb26726cc.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-af2f81d9c3eda3f1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-fe1f3bf513e17917.arrow


# Fine-tuning the model

In [22]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

2022-10-07 20:12:52.672117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 20:12:52.775554: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 20:12:53.136062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-07 20:12:53.136105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [23]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    num_train_epochs=24,
    
    weight_decay=0.01,
    
    save_strategy='epoch',
    save_total_limit = 3,
    
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
    
)

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [25]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

/tmp/ipykernel_1913209/1092872775.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [26]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'GPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [27]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
import torch
torch.cuda.empty_cache()

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 59924
  Num Epochs = 24
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 89904
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.079100,0.116812,0.809238,0.894441,0.849709,0.968354
2,0.057700,0.119632,0.812746,0.897624,0.853079,0.968575
3,0.043500,0.136282,0.818278,0.900652,0.857491,0.968973
4,0.033600,0.140391,0.817992,0.897352,0.855836,0.969289
5,0.026300,0.148464,0.826779,0.901118,0.862350,0.970177
6,0.022300,0.157671,0.822576,0.903913,0.861328,0.970022
7,0.015500,0.171137,0.822186,0.901506,0.860021,0.969598
8,0.012100,0.196515,0.817698,0.906204,0.859679,0.969160
9,0.009300,0.200058,0.820016,0.901196,0.858691,0.969555
10,0.008000,0.223168,0.821549,0.905816,0.861627,0.969910


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13900
  Batch size = 16
Saving model checkpoint to ../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-3746
Configuration saved in ../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-3746/config.json
Model weights saved in ../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-3746/pytorch_model.bin
tokenizer config file saved in ../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-3746/tokenizer_config.json
Special tokens file saved in ../data/models/microsoft/deberta-v3-small/SpeedOfMagic/ontonotes_english/checkpoint-3746/special_toke

TrainOutput(global_step=89904, training_loss=0.015481788217378301, metrics={'train_runtime': 4478.6067, 'train_samples_per_second': 321.121, 'train_steps_per_second': 20.074, 'total_flos': 1.997131024929091e+16, 'train_loss': 0.015481788217378301, 'epoch': 24.0})

In [31]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13900
  Batch size = 16
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it 

{'eval_loss': 0.2799668610095978,
 'eval_precision': 0.8291694800810263,
 'eval_recall': 0.9058156689183943,
 'eval_f1': 0.8657995806816706,
 'eval_accuracy': 0.9704236426623327,
 'eval_runtime': 14.1252,
 'eval_samples_per_second': 984.054,
 'eval_steps_per_second': 61.521,
 'epoch': 24.0}

In [32]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 13900
  Batch size = 16


{'CARDINAL': {'precision': 0.7664041994750657,
  'recall': 0.910467706013363,
  'f1': 0.8322475570032574,
  'number': 2245},
 'DATE': {'precision': 0.827538247566064,
  'recall': 0.8859440142942228,
  'f1': 0.8557457212713937,
  'number': 3358},
 'EVENT': {'precision': 0.6162162162162163,
  'recall': 0.5560975609756098,
  'f1': 0.5846153846153848,
  'number': 205},
 'FAC': {'precision': 0.4329004329004329,
  'recall': 0.6896551724137931,
  'f1': 0.5319148936170213,
  'number': 145},
 'GPE': {'precision': 0.9003185310099306,
  'recall': 0.9469846275128104,
  'f1': 0.9230621458073192,
  'number': 5074},
 'LANGUAGE': {'precision': 0.6875,
  'recall': 0.5789473684210527,
  'f1': 0.6285714285714286,
  'number': 38},
 'LAW': {'precision': 0.5679012345679012,
  'recall': 0.6666666666666666,
  'f1': 0.6133333333333333,
  'number': 69},
 'LOC': {'precision': 0.6917960088691796,
  'recall': 0.8062015503875969,
  'f1': 0.7446300715990453,
  'number': 387},
 'MONEY': {'precision': 0.91420534458509